In [1]:
from utils.ArticlesHandler import ArticlesHandler
from utils import solve, embedding_matrix_2_kNN, get_rate, accuracy, precision, recall, f1_score
from utils import Config
import time
import numpy as np
import scipy.sparse as sp
from postprocessing.SelectLabelsPostprocessor import SelectLabelsPostprocessor
from pygcn.utils import encode_onehot, accuracy, load_from_features
from model import AGNN
import torch
import torch.nn.functional as F
import torch.optim as optim

Import config file and check some values

In [2]:
config = Config(file='config')

assert (config.num_fake_articles + config.num_real_articles > 
        config.num_nearest_neighbours), "Can't have more neighbours than nodes!"

print("Method of decomposition:", config.method_decomposition_embedding)

Method of decomposition: GloVe


Import the articles and decompose the tensor.

In [3]:
print("Loading dataset", config.dataset_name)
articles = ArticlesHandler(config)

print("Performing decomposition...")
C = articles.get_tensor()

Loading dataset Random Poltical News Dataset
Performing decomposition...
Loading Glove Model
Done. 400000  words loaded!


Get the labels

In [4]:
config.set("num_unknown_labels", 195)

In [5]:
labels = articles.articles.labels
all_labels = articles.articles.labels_untouched

In [6]:
adj, features, all_labels = load_from_features(C, all_labels, config)
_, _, labels = load_from_features(C, labels, config)

In [7]:
# idx_train = range(150)
# idx_val = range(150, 175)
# idx_test = range(175, 200)
print(labels)
idx_train = np.where(labels)[0]
idx_val = np.where(1 - abs(labels))[0][:90]
idx_test = np.where(1 - abs(labels))[0][90:]

print(len(idx_train))

idx_train = torch.LongTensor(idx_train)
idx_val = torch.LongTensor(idx_val)
idx_test = torch.LongTensor(idx_test)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 2,
        1, 1, 2, 2, 2, 1])
10


In [8]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch.autograd import Variable
import torch.nn.functional as F


class GraphAttentionLayer(nn.Module):

    def __init__(self, requires_grad=True):
        super(GraphAttentionLayer, self).__init__()
        if requires_grad:
            # unifrom initialization
            self.beta = Parameter(torch.Tensor(1).uniform_(
                0, 1), requires_grad=requires_grad)
        else:
            self.beta = Variable(torch.zeros(1), requires_grad=requires_grad)

    def forward(self, x, adj):


        # NaN grad bug fixed at pytorch 0.3. Release note:
        #     `when torch.norm returned 0.0, the gradient was NaN.
        #     We now use the subgradient at 0.0, so the gradient is 0.0.`
        norm2 = torch.norm(x, 2, 1).view(-1, 1)

        # add a minor constant (1e-7) to denominator to prevent division by
        # zero error
        if torch.cuda.is_available():
            cos = self.beta.cuda() * \
                  torch.div(torch.mm(x, x.t()), torch.mm(norm2, norm2.t()) + 1e-7)
        else:
            cos = self.beta * \
                  torch.div(torch.mm(x, x.t()), torch.mm(norm2, norm2.t()) + 1e-7)

        # neighborhood masking (inspired by this repo:
        # https://github.com/danielegrattarola/keras-gat)
        
        
        mask = (torch.ones(adj.shape) - adj) * -1e9
        masked = cos + mask

        # propagation matrix
        P = F.softmax(masked, dim=1)

        # attention-guided propagation
        output = torch.mm(P, x)
        return output

    def __repr__(self):
        return self.__class__.__name__ + ' (16 -> 16)'


class LinearLayer(nn.Module):

    def __init__(self, in_features, out_features, initializer=nn.init.xavier_uniform_):
        super(LinearLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(initializer(
            torch.Tensor(in_features, out_features)))

    def forward(self, input):
        # no bias
        return torch.mm(input, self.weight)

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
            + str(self.in_features) + ' -> ' \
            + str(self.out_features) + ')'


class AGNN(nn.Module):

    def __init__(self, nfeat, nhid, nclass, nlayers, dropout_rate):
        super(AGNN, self).__init__()

        self.layers = nlayers
        self.dropout_rate = dropout_rate

        self.embeddinglayer = LinearLayer(nfeat, nhid)
        nn.init.xavier_uniform_(self.embeddinglayer.weight)

        self.attentionlayers = nn.ModuleList()
        # for Cora dataset, the first propagation layer is non-trainable
        # and beta is fixed at 0
        self.attentionlayers.append(GraphAttentionLayer(requires_grad=False).cuda())
        for i in range(1, self.layers):
            if torch.cuda.is_available():
                self.attentionlayers.append(GraphAttentionLayer().cuda())
            else:
                self.attentionlayers.append(GraphAttentionLayer())


        self.outputlayer = LinearLayer(nhid, nclass)
        nn.init.xavier_uniform_(self.outputlayer.weight)

    def forward(self, x, adj):
        x = F.relu(self.embeddinglayer(x))
        x = F.dropout(x, self.dropout_rate, training=self.training)

        for i in range(self.layers):
            x = self.attentionlayers[i](x, adj)

        x = self.outputlayer(x)
        x = F.dropout(x, self.dropout_rate, training=self.training)
        return F.log_softmax(x, dim=1)

In [10]:
cuda = False
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
fastmode = False
epochs = 900#450
layers =2

# Model and optimizer
model = AGNN(nfeat=features.shape[1],
                     nhid=hidden,
                     nclass=2,
                     nlayers=layers,
                     dropout_rate=0.5)

optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    all_labels = all_labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)

    
    loss_train = F.nll_loss(output[idx_train], all_labels[idx_train])
    acc_train = accuracy(output[idx_train], all_labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], all_labels[idx_val])
    acc_val = accuracy(output[idx_val], all_labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], all_labels[idx_test])
    acc_test = accuracy(output[idx_test], all_labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 0001 loss_train: 0.7236 acc_train: 0.4000 loss_val: 0.6892 acc_val: 0.5222 time: 0.0105s
Epoch: 0002 loss_train: 0.7658 acc_train: 0.5000 loss_val: 0.6851 acc_val: 0.5222 time: 0.0076s
Epoch: 0003 loss_train: 0.6747 acc_train: 0.6000 loss_val: 0.6866 acc_val: 0.4778 time: 0.0092s
Epoch: 0004 loss_train: 0.7075 acc_train: 0.5000 loss_val: 0.6940 acc_val: 0.5000 time: 0.0095s
Epoch: 0005 loss_train: 0.6940 acc_train: 0.5000 loss_val: 0.7002 acc_val: 0.4778 time: 0.0146s
Epoch: 0006 loss_train: 0.6244 acc_train: 0.7000 loss_val: 0.7039 acc_val: 0.4778 time: 0.0124s
Epoch: 0007 loss_train: 0.6725 acc_train: 0.7000 loss_val: 0.6983 acc_val: 0.4778 time: 0.0059s
Epoch: 0008 loss_train: 0.7848 acc_train: 0.4000 loss_val: 0.6908 acc_val: 0.5111 time: 0.0058s
Epoch: 0009 loss_train: 0.6623 acc_train: 0.5000 loss_val: 0.6897 acc_val: 0.5667 time: 0.0057s
Epoch: 0010 loss_train: 0.6590 acc_train: 0.7000 loss_val: 0.6860 acc_val: 0.7000 time: 0.0055s
Epoch: 0011 loss_train: 0.6658 acc_train

Epoch: 0089 loss_train: 0.4046 acc_train: 0.7000 loss_val: 0.7966 acc_val: 0.6111 time: 0.0080s
Epoch: 0090 loss_train: 0.4104 acc_train: 0.8000 loss_val: 0.7928 acc_val: 0.6222 time: 0.0083s
Epoch: 0091 loss_train: 0.3830 acc_train: 0.8000 loss_val: 0.7970 acc_val: 0.6222 time: 0.0076s
Epoch: 0092 loss_train: 0.4092 acc_train: 0.7000 loss_val: 0.8061 acc_val: 0.6111 time: 0.0094s
Epoch: 0093 loss_train: 0.3110 acc_train: 0.8000 loss_val: 0.8087 acc_val: 0.6111 time: 0.0069s
Epoch: 0094 loss_train: 0.3236 acc_train: 0.7000 loss_val: 0.8125 acc_val: 0.6111 time: 0.0063s
Epoch: 0095 loss_train: 0.2793 acc_train: 0.8000 loss_val: 0.8159 acc_val: 0.6222 time: 0.0116s
Epoch: 0096 loss_train: 0.3084 acc_train: 0.9000 loss_val: 0.8210 acc_val: 0.6222 time: 0.0055s
Epoch: 0097 loss_train: 0.2635 acc_train: 0.9000 loss_val: 0.8301 acc_val: 0.6111 time: 0.0056s
Epoch: 0098 loss_train: 0.3799 acc_train: 0.7000 loss_val: 0.8364 acc_val: 0.6111 time: 0.0057s
Epoch: 0099 loss_train: 0.2703 acc_train

Epoch: 0181 loss_train: 0.2347 acc_train: 0.8000 loss_val: 1.1408 acc_val: 0.6111 time: 0.0103s
Epoch: 0182 loss_train: 0.1958 acc_train: 0.8000 loss_val: 1.1474 acc_val: 0.6333 time: 0.0083s
Epoch: 0183 loss_train: 0.3515 acc_train: 0.8000 loss_val: 1.1449 acc_val: 0.6111 time: 0.0074s
Epoch: 0184 loss_train: 0.2481 acc_train: 0.8000 loss_val: 1.1443 acc_val: 0.6111 time: 0.0066s
Epoch: 0185 loss_train: 0.3830 acc_train: 0.7000 loss_val: 1.1447 acc_val: 0.6222 time: 0.0065s
Epoch: 0186 loss_train: 0.2288 acc_train: 0.9000 loss_val: 1.1536 acc_val: 0.6111 time: 0.0114s
Epoch: 0187 loss_train: 0.3337 acc_train: 0.9000 loss_val: 1.1647 acc_val: 0.6111 time: 0.0071s
Epoch: 0188 loss_train: 0.2674 acc_train: 1.0000 loss_val: 1.1825 acc_val: 0.6111 time: 0.0068s
Epoch: 0189 loss_train: 0.3735 acc_train: 0.8000 loss_val: 1.2016 acc_val: 0.6222 time: 0.0058s
Epoch: 0190 loss_train: 0.3038 acc_train: 0.7000 loss_val: 1.2124 acc_val: 0.6111 time: 0.0062s
Epoch: 0191 loss_train: 0.3558 acc_train

Epoch: 0282 loss_train: 0.1623 acc_train: 0.9000 loss_val: 1.3700 acc_val: 0.6222 time: 0.0135s
Epoch: 0283 loss_train: 0.2939 acc_train: 0.8000 loss_val: 1.3745 acc_val: 0.6111 time: 0.0121s
Epoch: 0284 loss_train: 0.4117 acc_train: 0.8000 loss_val: 1.3857 acc_val: 0.6111 time: 0.0076s
Epoch: 0285 loss_train: 0.3059 acc_train: 0.8000 loss_val: 1.4071 acc_val: 0.6111 time: 0.0110s
Epoch: 0286 loss_train: 0.1749 acc_train: 1.0000 loss_val: 1.4287 acc_val: 0.6111 time: 0.0080s
Epoch: 0287 loss_train: 0.2335 acc_train: 1.0000 loss_val: 1.4580 acc_val: 0.6000 time: 0.0081s
Epoch: 0288 loss_train: 0.1136 acc_train: 1.0000 loss_val: 1.4671 acc_val: 0.6000 time: 0.0071s
Epoch: 0289 loss_train: 0.2085 acc_train: 0.9000 loss_val: 1.4638 acc_val: 0.6000 time: 0.0072s
Epoch: 0290 loss_train: 0.1066 acc_train: 0.9000 loss_val: 1.4378 acc_val: 0.6111 time: 0.0069s
Epoch: 0291 loss_train: 0.2265 acc_train: 0.9000 loss_val: 1.4175 acc_val: 0.6111 time: 0.0072s
Epoch: 0292 loss_train: 0.3256 acc_train

Epoch: 0395 loss_train: 0.4873 acc_train: 0.8000 loss_val: 1.5428 acc_val: 0.6111 time: 0.0100s
Epoch: 0396 loss_train: 0.4078 acc_train: 0.7000 loss_val: 1.5488 acc_val: 0.6111 time: 0.0076s
Epoch: 0397 loss_train: 0.0951 acc_train: 1.0000 loss_val: 1.5495 acc_val: 0.6111 time: 0.0120s
Epoch: 0398 loss_train: 0.3214 acc_train: 0.8000 loss_val: 1.5492 acc_val: 0.6111 time: 0.0168s
Epoch: 0399 loss_train: 0.2426 acc_train: 0.9000 loss_val: 1.5445 acc_val: 0.6222 time: 0.0125s
Epoch: 0400 loss_train: 0.1833 acc_train: 0.9000 loss_val: 1.5407 acc_val: 0.6111 time: 0.0076s
Epoch: 0401 loss_train: 0.2191 acc_train: 0.8000 loss_val: 1.5372 acc_val: 0.6111 time: 0.0100s
Epoch: 0402 loss_train: 0.3851 acc_train: 0.8000 loss_val: 1.5401 acc_val: 0.6111 time: 0.0129s
Epoch: 0403 loss_train: 0.1156 acc_train: 1.0000 loss_val: 1.5453 acc_val: 0.6111 time: 0.0087s
Epoch: 0404 loss_train: 0.2813 acc_train: 0.7000 loss_val: 1.5494 acc_val: 0.6111 time: 0.0090s
Epoch: 0405 loss_train: 0.1028 acc_train

Epoch: 0485 loss_train: 0.1953 acc_train: 0.8000 loss_val: 1.6818 acc_val: 0.5778 time: 0.0098s
Epoch: 0486 loss_train: 0.1909 acc_train: 0.9000 loss_val: 1.6910 acc_val: 0.5778 time: 0.0110s
Epoch: 0487 loss_train: 0.4887 acc_train: 0.7000 loss_val: 1.6648 acc_val: 0.5778 time: 0.0135s
Epoch: 0488 loss_train: 0.3162 acc_train: 0.9000 loss_val: 1.6241 acc_val: 0.5889 time: 0.0069s
Epoch: 0489 loss_train: 0.2316 acc_train: 0.9000 loss_val: 1.5911 acc_val: 0.5778 time: 0.0112s
Epoch: 0490 loss_train: 0.2298 acc_train: 0.9000 loss_val: 1.5629 acc_val: 0.6111 time: 0.0075s
Epoch: 0491 loss_train: 0.2531 acc_train: 0.9000 loss_val: 1.5460 acc_val: 0.6111 time: 0.0108s
Epoch: 0492 loss_train: 0.2312 acc_train: 1.0000 loss_val: 1.5408 acc_val: 0.6111 time: 0.0057s
Epoch: 0493 loss_train: 0.3344 acc_train: 0.8000 loss_val: 1.5469 acc_val: 0.6111 time: 0.0109s
Epoch: 0494 loss_train: 0.2961 acc_train: 0.9000 loss_val: 1.5709 acc_val: 0.6000 time: 0.0055s
Epoch: 0495 loss_train: 0.2659 acc_train

Epoch: 0580 loss_train: 0.2392 acc_train: 0.9000 loss_val: 1.6709 acc_val: 0.5778 time: 0.0084s
Epoch: 0581 loss_train: 0.3423 acc_train: 0.7000 loss_val: 1.6772 acc_val: 0.5889 time: 0.0073s
Epoch: 0582 loss_train: 0.2229 acc_train: 0.9000 loss_val: 1.6769 acc_val: 0.5889 time: 0.0092s
Epoch: 0583 loss_train: 0.2647 acc_train: 0.8000 loss_val: 1.6723 acc_val: 0.5889 time: 0.0075s
Epoch: 0584 loss_train: 0.2812 acc_train: 0.8000 loss_val: 1.6503 acc_val: 0.5778 time: 0.0079s
Epoch: 0585 loss_train: 0.2642 acc_train: 0.9000 loss_val: 1.6317 acc_val: 0.6111 time: 0.0070s
Epoch: 0586 loss_train: 0.1157 acc_train: 0.9000 loss_val: 1.6232 acc_val: 0.6111 time: 0.0071s
Epoch: 0587 loss_train: 0.1661 acc_train: 0.8000 loss_val: 1.6277 acc_val: 0.6111 time: 0.0065s
Epoch: 0588 loss_train: 0.1972 acc_train: 0.9000 loss_val: 1.6407 acc_val: 0.6111 time: 0.0083s
Epoch: 0589 loss_train: 0.2750 acc_train: 0.7000 loss_val: 1.6569 acc_val: 0.6111 time: 0.0058s
Epoch: 0590 loss_train: 0.0985 acc_train

Epoch: 0678 loss_train: 0.3188 acc_train: 0.8000 loss_val: 1.6957 acc_val: 0.5778 time: 0.0155s
Epoch: 0679 loss_train: 0.2225 acc_train: 0.8000 loss_val: 1.6862 acc_val: 0.5778 time: 0.0142s
Epoch: 0680 loss_train: 0.2254 acc_train: 0.9000 loss_val: 1.6826 acc_val: 0.5778 time: 0.0137s
Epoch: 0681 loss_train: 0.2905 acc_train: 0.9000 loss_val: 1.6711 acc_val: 0.5778 time: 0.0114s
Epoch: 0682 loss_train: 0.4520 acc_train: 0.7000 loss_val: 1.6580 acc_val: 0.5778 time: 0.0069s
Epoch: 0683 loss_train: 0.1609 acc_train: 0.9000 loss_val: 1.6219 acc_val: 0.6000 time: 0.0107s
Epoch: 0684 loss_train: 0.1381 acc_train: 0.9000 loss_val: 1.6006 acc_val: 0.5778 time: 0.0069s
Epoch: 0685 loss_train: 0.3511 acc_train: 0.8000 loss_val: 1.5769 acc_val: 0.5889 time: 0.0110s
Epoch: 0686 loss_train: 0.1282 acc_train: 1.0000 loss_val: 1.5554 acc_val: 0.6111 time: 0.0085s
Epoch: 0687 loss_train: 0.2556 acc_train: 0.9000 loss_val: 1.5466 acc_val: 0.6111 time: 0.0079s
Epoch: 0688 loss_train: 0.2413 acc_train

Epoch: 0766 loss_train: 0.4119 acc_train: 0.8000 loss_val: 1.4562 acc_val: 0.5778 time: 0.0073s
Epoch: 0767 loss_train: 0.2085 acc_train: 0.8000 loss_val: 1.4610 acc_val: 0.6111 time: 0.0127s
Epoch: 0768 loss_train: 0.1580 acc_train: 1.0000 loss_val: 1.4672 acc_val: 0.6111 time: 0.0104s
Epoch: 0769 loss_train: 0.2764 acc_train: 0.9000 loss_val: 1.4711 acc_val: 0.5778 time: 0.0064s
Epoch: 0770 loss_train: 0.2296 acc_train: 0.8000 loss_val: 1.4776 acc_val: 0.5778 time: 0.0094s
Epoch: 0771 loss_train: 0.4842 acc_train: 0.5000 loss_val: 1.4790 acc_val: 0.5778 time: 0.0061s
Epoch: 0772 loss_train: 0.2891 acc_train: 0.9000 loss_val: 1.4654 acc_val: 0.5778 time: 0.0062s
Epoch: 0773 loss_train: 0.2706 acc_train: 0.9000 loss_val: 1.4553 acc_val: 0.5778 time: 0.0063s
Epoch: 0774 loss_train: 0.2914 acc_train: 0.7000 loss_val: 1.4475 acc_val: 0.6111 time: 0.0062s
Epoch: 0775 loss_train: 0.2659 acc_train: 0.9000 loss_val: 1.4516 acc_val: 0.5778 time: 0.0063s
Epoch: 0776 loss_train: 0.2318 acc_train

Epoch: 0857 loss_train: 0.1045 acc_train: 1.0000 loss_val: 1.3271 acc_val: 0.5889 time: 0.0082s
Epoch: 0858 loss_train: 0.3702 acc_train: 0.7000 loss_val: 1.3048 acc_val: 0.5778 time: 0.0136s
Epoch: 0859 loss_train: 0.2537 acc_train: 0.9000 loss_val: 1.2837 acc_val: 0.5778 time: 0.0154s
Epoch: 0860 loss_train: 0.1729 acc_train: 0.9000 loss_val: 1.2670 acc_val: 0.6111 time: 0.0124s
Epoch: 0861 loss_train: 0.1604 acc_train: 0.9000 loss_val: 1.2397 acc_val: 0.6111 time: 0.0058s
Epoch: 0862 loss_train: 0.3298 acc_train: 0.7000 loss_val: 1.1879 acc_val: 0.6111 time: 0.0122s
Epoch: 0863 loss_train: 0.2320 acc_train: 0.9000 loss_val: 1.1288 acc_val: 0.5889 time: 0.0092s
Epoch: 0864 loss_train: 0.2240 acc_train: 0.9000 loss_val: 1.0758 acc_val: 0.5778 time: 0.0091s
Epoch: 0865 loss_train: 0.3636 acc_train: 0.8000 loss_val: 1.0377 acc_val: 0.5667 time: 0.0058s
Epoch: 0866 loss_train: 0.3659 acc_train: 0.8000 loss_val: 1.0155 acc_val: 0.5667 time: 0.0058s
Epoch: 0867 loss_train: 0.1819 acc_train